# Stock portfolio optimizer

According to Wikipedia, portfolio optimization is the process of selecting the best portfolio out of the set of all portfolios being considered according to some objective. The objective typically maximizes factors such as expected returns and minizes cost like financial risk.

The program that I will create will optimizes a stock portfolio. It will recommend the number of stocks to buy with the amount of money being invested in the portfolio. The algorithm will choose how many stocks to own for certain companies based on the values of expected annual return, annual volatility, and sharpe ratio. The amount of money remaining after investing in the portfolio will also be shown.

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import yfinance as yf

In [ ]:
tickers_list = ['AAPL', 'WMT', 'IBM', 'MU', 'BA', 'AXP', 'TSLA', 'ETSY',
                'NVDA', 'PYPL', 'FCX', 'MMM', 'ACN', 'ADBE','GOOG']


In [ ]:
df = yf.download(tickers_list,'2015-1-1')['Adj Close']

In [ ]:
df

,AAPL,ACN,ADBE,AXP,BA,ETSY,FCX,GOOG,IBM,MMM,MU,NVDA,PYPL,TSLA,WMT
Date,,,,,,,,,,,,,,,
2015-01-02,24.861401,79.502106,72.339996,84.234779,113.657227,NaN,22.150909,523.373108,124.574829,137.071991,34.750000,19.374790,NaN,43.862000,74.555740
2015-01-05,24.161013,78.159752,71.980003,82.007118,112.870056,NaN,20.834080,512.463013,122.614647,133.980652,33.779999,19.047548,NaN,42.018002,74.338768
2015-01-06,24.163294,77.595978,70.529999,80.259384,111.540627,NaN,21.191505,500.585632,119.970337,132.551971,32.869999,18.470057,NaN,42.256001,74.911591
2015-01-07,24.502111,79.224663,71.110001,82.012276,113.272369,NaN,21.501902,499.727997,119.186256,133.512802,32.099998,18.421932,NaN,42.189999,76.899170
2015-01-08,25.443539,80.432770,72.919998,83.174782,115.275284,NaN,21.981598,501.303680,121.776733,136.712753,33.669998,19.114916,NaN,42.124001,78.522209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-16,133.190002,257.049988,501.640015,130.679993,217.179993,228.320007,32.900002,2121.899902,120.070000,176.630005,87.739998,613.210022,304.790009,796.219971,145.660004
2021-02-17,130.839996,258.779999,491.230011,128.559998,215.520004,222.410004,34.150002,2128.310059,119.970001,176.649994,86.050003,596.239990,297.200012,798.150024,147.199997
2021-02-18,129.710007,257.630005,488.369995,128.490005,208.479996,220.820007,34.139999,2117.199951,120.730003,179.369995,88.540001,593.159973,290.809998,787.380005,137.660004


In [ ]:
#Get the assets /tickers
assets = df.columns

In [ ]:
#Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
#Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(df) #Expected returns
S = risk_models.sample_cov(df) #Sample covariance matrix

In [ ]:
#Optimize for the maximal Sharpe ratio
#Sharpe ratio describes how much excess return you receive for the extra volatility you endure for holding a riskier asset
ef = EfficientFrontier(mu, S) #Create the Efficient Frontier Object
weights = ef.max_sharpe() #Maximize the sharpe ratio and get the raw weights

cleaned_weights = ef.clean_weights() #Helper method to clean the raw weights, setting any weights whose absolute value are below the cutoff of 0 and rounding the rest(can cause rounding errors) 
print(cleaned_weights) 

#Show expected returns, volatility and sharpe ratio
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('AXP', 0.0), ('BA', 0.0), ('ETSY', 0.00919), ('FCX', 0.0), ('GOOG', 0.0), ('IBM', 0.0), ('MMM', 0.0), ('MU', 0.0), ('NVDA', 0.59004), ('PYPL', 0.2282), ('TSLA', 0.17257), ('WMT', 0.0)])
Expected annual return: 63.7%
Annual volatility: 36.7%
Sharpe Ratio: 1.68


(0.6370229301167718, 0.3674471069330211, 1.6792156434891834)

Sharpe ratio that is greater than 1.0 are considered acceptable to investors.

In [ ]:
#Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

#Money in USD that is going to be put in the portfolio
portfolio_val = 5000
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices,total_portfolio_value = portfolio_val)
allocation, leftover = da.lp_portfolio() #Store stock allocation and amount that is remaining
print('Discrete allocation:', allocation)
print('Funds Remaining: $', leftover)

Discrete allocation: {'ETSY': 1, 'NVDA': 5, 'PYPL': 4, 'TSLA': 1}
Funds Remaining: $ 105.830078125


In [ ]:
#Create a function to get the companies name
def get_company_name(symbol):
  url = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query=' + symbol + '&region=1&lang=en'
  result = requests.get(url).json()
  for r in result['ResultSet']['Result']:
    if r['symbol']==symbol:
      return r['name']

In [ ]:
#Store the company name into a list
company_name = []
for symbol in allocation:
  company_name.append( get_company_name(symbol))

In [ ]:
#Get the discrete allocation values
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [ ]:
#Create a dataframe for the portfolio
portfolio_df = pd.DataFrame(columns = ['Company_name', 'Company_Ticker', 'Discrete_val_' +str(portfolio_val)])

In [ ]:
portfolio_df['Company_name'] = company_name
portfolio_df['Company_Ticker'] = allocation
portfolio_df['Discrete_val_'+str(portfolio_val)] = discrete_allocation_list

In [ ]:
#Show the portfolio
portfolio_df

,Company_name,Company_Ticker,Discrete_val_5000
0,"Etsy, Inc.",ETSY,1
1,NVIDIA Corporation,NVDA,5
2,"PayPal Holdings, Inc.",PYPL,4
3,"Tesla, Inc.",TSLA,1
